# Othello Computer Vision Analysis

In [1]:
import cv2
import numpy as np


"""
Connect 4
X = blue = player 1
O = red = player 2

Othello
X = B = black = player 1
O = W = white = player 2
"""

BOARD_WIDTH = 4
BOARD_HEIGHT = 4

def grid_to_position_string(grid):
    rows, cols = grid.shape  # Assuming grid is a numpy array with .shape attribute
    position_string = ''

    for col in range(cols):
        for row in range(rows):
            if grid[row][col] == 1:
                position_string += 'B'
            elif grid[row][col] == -1:
                position_string += 'W'
            else:
                position_string += '-'

    return position_string

# Main
def process_frame(frame):
    img = frame

    # Constants
    new_width = 500
    img_h, img_w, _ = img.shape
    scale = new_width / img_w
    img_w = int(img_w * scale)
    img_h = int(img_h * scale)
    img = cv2.resize(img, (img_w, img_h), interpolation=cv2.INTER_AREA)
    img_orig = img.copy()

    # Bilateral Filter
    bilateral_filtered_image = cv2.bilateralFilter(img, 15, 190, 190)
    # cv2.imwrite('bilateral_filtered_image.png', bilateral_filtered_image)

    # Calculate the size of each grid cell
    cell_width = img_w // BOARD_WIDTH
    cell_height = img_h // BOARD_HEIGHT

    # Create a copy of the original image to draw the grid
    grid_image = img_orig.copy()

    # Draw vertical lines for the grid
    for i in range(1, BOARD_WIDTH):
        cv2.line(grid_image, (i * cell_width, 0), (i * cell_width, img_h), (0, 255, 0), 1)

    # Draw horizontal lines for the grid
    for i in range(1, BOARD_HEIGHT):
        cv2.line(grid_image, (0, i * cell_height), (img_w, i * cell_height), (0, 255, 0), 1)

    # Display the image with the grid
    cv2.imwrite('grid_image_with_cells.png', grid_image)

    # Initialize the grid
    grid = np.zeros((BOARD_HEIGHT, BOARD_WIDTH))

    # Constants for color detection for black and white pieces RGB bounds!
    BLACK_LOWER_HSV = np.array([0, 0, 0])
    BLACK_UPPER_HSV = np.array([110, 110, 110])
    WHITE_LOWER_HSV = np.array([150, 150, 150])
    WHITE_UPPER_HSV = np.array([255, 255, 255])

    # Function to check if the majority of the pixels in a masked area are of the color of the mask
    def is_color_dominant(mask):
        # Count the non-zero (white) pixels in the mask
        white_pixels = cv2.countNonZero(mask)
        # Calculate the percentage of white pixels
        white_area_ratio = white_pixels / mask.size
        # If the white area covers more than 30% of the mask, we consider the color to be dominant
        return white_area_ratio > 0.3


    def process_cell(img, x_start, y_start, width, height):
        # Crop the cell from the image
        cell_img = img[y_start:y_start + height, x_start:x_start + width]
        cv2.imwrite('cell_img.png', cell_img)

        # Create masks for white and black pieces
        white_mask_cell = cv2.inRange(cell_img, WHITE_LOWER_HSV, WHITE_UPPER_HSV)
        cv2.imwrite('white_mask_cell.png', white_mask_cell)
        black_mask_cell = cv2.inRange(cell_img, BLACK_LOWER_HSV, BLACK_UPPER_HSV)
        cv2.imwrite('black_mask_cell.png', black_mask_cell)

        if is_color_dominant(white_mask_cell):
            print("White piece detected")
            return -1  # White 
        elif is_color_dominant(black_mask_cell):
            print("Black piece detected")
            return 1  # Black 
        else:
            # The space is empty
            return 0
        
        
    # CONTINUE PROCESS FRAME
    # Analyze each cell and update the grid
    for row in range(BOARD_HEIGHT):
        for col in range(BOARD_WIDTH):
            x_start = col * cell_width
            y_start = row * cell_height
            grid[row, col] = process_cell(bilateral_filtered_image, x_start, y_start, cell_width, cell_height)

    
    return grid

# determines if there is motion in the frame
backSub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)

# Could be better...
def is_motion(previous_frame, current_frame, threshold=10):
    frame_delta = cv2.absdiff(previous_frame, current_frame)
    thresholded = cv2.threshold(frame_delta, 25, 255, cv2.THRESH_BINARY)[1]
    motion_level = np.sum(thresholded)
    return motion_level > threshold


# Need to compelte find_board_bounds
def find_board_bounds(frame, threshold=50):
    x, y = 0, 0  # Top left corner
    w, h = 1154, 1152  # Width and height calculated from bottom right - top left
    
    return x, y, w, h


def board_is_full(grid):
    for row in grid:
        for cell in row:
            if cell == 0:
                return False
    return True


def check_winner(grid):
    rows, cols = len(grid), len(grid[0])
    grid_array = np.array(grid)

    # If the board is not full then we have not found a winner
    if (not board_is_full(grid_array)):
        return 0
    
    # If the board is full then we need to check for a winner
    
    # Win condition
    # 1. count the number of black and white tiles
    # 2. return the color & count with more tiles

    black_count = 0
    white_count = 0
    for row in grid_array:
        for cell in grid_array:
            if cell == 1:
                black_count += 1
            elif cell == -1:
                white_count += 1

    # If black has more tiles then black wins
    if black_count > white_count:
        return 1
    elif white_count > black_count:
        return -1
    else:
        return 0


def extract_frames(video_path, skip_frames=20):
    cap = cv2.VideoCapture(video_path)
    previous_position_string = None
    output_strings = []  # List to store output strings

    ret, previous_frame = cap.read()
    if not ret:
        output_strings.append("Error: Cannot read frame from video.")
        cap.release()
        return output_strings
    
    previous_frame = cv2.cvtColor(previous_frame, cv2.COLOR_BGR2GRAY)
    previous_frame = cv2.GaussianBlur(previous_frame, (21, 21), 0)
    previous_position_string = '------------------------------------------'
    p, frame_count = 0, 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)
        if frame_count % skip_frames == 0:
            if not is_motion(previous_frame, gray):
                board_array = process_frame(frame)
                current_position_string = grid_to_position_string(board_array)
                
                # There is a change between one game state and the next
                if current_position_string != previous_position_string:
                    output_strings.append(f'p={str((p % 2) + 1)}_{current_position_string}')
                    p += 1

                previous_position_string = current_position_string
            previous_frame = gray
        frame_count += 1

    cap.release()
    return output_strings

print(extract_frames('uploads/othellogameplay.mov'))

Black piece detected
White piece detected
White piece detected
Black piece detected
Black piece detected
White piece detected
White piece detected
Black piece detected
Black piece detected
White piece detected
White piece detected
Black piece detected
Black piece detected
White piece detected
White piece detected
Black piece detected
Black piece detected
Black piece detected
Black piece detected
White piece detected
Black piece detected
Black piece detected
Black piece detected
Black piece detected
White piece detected
Black piece detected
Black piece detected
Black piece detected
Black piece detected
White piece detected
Black piece detected
White piece detected
Black piece detected
White piece detected
Black piece detected
White piece detected
Black piece detected
White piece detected
Black piece detected
White piece detected
Black piece detected
White piece detected
Black piece detected
White piece detected
Black piece detected
White piece detected
Black piece detected
White piece d

In [15]:
print(extract_frames('uploads/othello_gamesman_uni_cropped.mp4'))

White piece detected
Black piece detected
Black piece detected
White piece detected
White piece detected
Black piece detected
Black piece detected
White piece detected
White piece detected
Black piece detected
Black piece detected
White piece detected
White piece detected
Black piece detected
Black piece detected
White piece detected
White piece detected
Black piece detected
Black piece detected
White piece detected
White piece detected
Black piece detected
Black piece detected
White piece detected
Black piece detected
White piece detected
Black piece detected
Black piece detected
Black piece detected
Black piece detected
Black piece detected
Black piece detected
White piece detected
White piece detected
White piece detected
White piece detected
White piece detected
White piece detected
Black piece detected
Black piece detected
White piece detected
Black piece detected
White piece detected
Black piece detected
White piece detected
White piece detected
White piece detected
White piece d

# Perspective Transformation

In [18]:
import cv2
import numpy as np

# Function to detect corners in the image
def detect_corners(frame):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect corners using the Harris corner detector
    corners = cv2.cornerHarris(gray, 2, 3, 0.04)
    
    # Normalize the corner response to lie between 0 and 255
    cv2.normalize(corners, corners, 0, 255, cv2.NORM_MINMAX)
    
    # Threshold the corner response to retain only strong corners
    threshold = 100
    corner_mask = np.uint8(corners > threshold)
    
    # Dilate the corner points to make them more visible
    corner_mask = cv2.dilate(corner_mask, None)
    
    # Find contours of the corner points
    contours, _ = cv2.findContours(corner_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw the detected corners on the original frame
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    return frame, contours

# Function to perform perspective transform
def perspective_transform(frame, contours):
    # Compute the centroid of each contour and sort them based on their x-coordinate
    centroids = [(int(cv2.moments(cnt)["m10"] / cv2.moments(cnt)["m00"]),
                  int(cv2.moments(cnt)["m01"] / cv2.moments(cnt)["m00"])) for cnt in contours]
    
    # Sort the centroids based on their x-coordinate
    sorted_centroids = sorted(centroids, key=lambda x: x[0])
    
    # Ensure that there are at least four centroids detected
    if len(sorted_centroids) < 4:
        print("Error: Less than four corners detected.")
        return frame
    
    # Extract the corner points from the sorted centroids
    src_pts = np.float32([sorted_centroids[0], sorted_centroids[1], sorted_centroids[-1], sorted_centroids[-2]])
    
    # Define the destination points for perspective transform
    dest_pts = np.float32([[0, 0], [400, 0], [400, 400], [0, 400]])
    
    # Compute the perspective transform matrix
    transform_matrix = cv2.getPerspectiveTransform(src_pts, dest_pts)
    
    # Apply the perspective transform
    transformed_frame = cv2.warpPerspective(frame, transform_matrix, (400, 400))
    
    return transformed_frame


# Function to process the frame with existing code
def process_frame(frame):
    # Your existing code to process the frame goes here
    # For demonstration, we'll just return the input frame
    return frame

# Main function
def main(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Detect corners in the frame
        frame_with_corners, corners = detect_corners(frame.copy())
        
        # Perform perspective transform
        transformed_frame = perspective_transform(frame.copy(), corners)
        
        # Process the transformed frame with existing code
        processed_frame = process_frame(transformed_frame)
        
        # Display the processed frame
        cv2.imshow("Processed Frame", processed_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Run the main function with the path to your video
main('uploads/othello_real_world_wiki.mov')

Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.
Error: Less than four corners detected.


: 

# Perspective Transform with Video Export Feature

In [12]:
import cv2
import numpy as np

# Function to detect corners in the image
def detect_corners(frame):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect corners using the Harris corner detector
    corners = cv2.cornerHarris(gray, 2, 3, 0.04)
    
    # Normalize the corner response to lie between 0 and 255
    cv2.normalize(corners, corners, 0, 255, cv2.NORM_MINMAX)
    
    # Threshold the corner response to retain only strong corners
    threshold = 100
    corner_mask = np.uint8(corners > threshold)
    
    # Dilate the corner points to make them more visible
    corner_mask = cv2.dilate(corner_mask, None)
    
    # Find contours of the corner points
    contours, _ = cv2.findContours(corner_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw the detected corners on the original frame
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    return frame, contours

# Function to perform perspective transform
def perspective_transform(frame, contours):
    # Compute the centroid of each contour and sort them based on their x-coordinate
    centroids = [(int(cv2.moments(cnt)["m10"] / cv2.moments(cnt)["m00"]),
                  int(cv2.moments(cnt)["m01"] / cv2.moments(cnt)["m00"])) for cnt in contours]
    sorted_centroids = sorted(centroids, key=lambda x: x[0])
    
    # Extract the corner points from the sorted centroids
    src_pts = np.float32([sorted_centroids[0], sorted_centroids[1], sorted_centroids[-1], sorted_centroids[-2]])
    
    # Define the destination points for perspective transform
    dest_pts = np.float32([[0, 0], [400, 0], [400, 400], [0, 400]])
    
    # Compute the perspective transform matrix
    transform_matrix = cv2.getPerspectiveTransform(src_pts, dest_pts)
    
    # Apply the perspective transform
    transformed_frame = cv2.warpPerspective(frame, transform_matrix, (400, 400))
    
    return transformed_frame


# Function to process the frame with existing code
def process_frame(frame):
    # Your existing code to process the frame goes here
    # For demonstration, we'll just return the input frame
    return frame

# Main function
def main(video_path, output_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (400, 400))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Detect corners in the frame
        frame_with_corners, corners = detect_corners(frame.copy())
        
        # Perform perspective transform
        transformed_frame = perspective_transform(frame.copy(), corners)
        
        # Process the transformed frame with existing code
        processed_frame = process_frame(transformed_frame)
        
        # Display the processed frame
        cv2.imshow("Processed Frame", processed_frame)
        
        # Write the processed frame to the output video file
        out.write(processed_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release everything if job is finished
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Run the main function with the path to your video and output video path
main('uploads/gamesmanuni_complicated.mov', 'output_video.avi')

# Othello Computer Vision with Images Displayed

In [4]:
import cv2
import numpy as np


"""
Connect 4
X = blue = player 1
O = red = player 2

Othello
X = B = black = player 1
O = W = white = player 2
"""

BOARD_WIDTH = 4
BOARD_HEIGHT = 4

def grid_to_position_string(grid):
    rows, cols = grid.shape  # Assuming grid is a numpy array with .shape attribute
    position_string = ''

    for col in range(cols):
        for row in range(rows):
            if grid[row][col] == 1:
                position_string += 'B'
            elif grid[row][col] == -1:
                position_string += 'W'
            else:
                position_string += '-'

    return position_string

# Main
def process_frame(frame):
    img = frame

    # Constants
    new_width = 500
    img_h, img_w, _ = img.shape
    scale = new_width / img_w
    img_w = int(img_w * scale)
    img_h = int(img_h * scale)
    img = cv2.resize(img, (img_w, img_h), interpolation=cv2.INTER_AREA)
    img_orig = img.copy()

    # Bilateral Filter
    bilateral_filtered_image = cv2.bilateralFilter(img, 15, 190, 190)
    # cv2.imwrite('bilateral_filtered_image.png', bilateral_filtered_image)

    # Calculate the size of each grid cell
    cell_width = img_w // BOARD_WIDTH
    cell_height = img_h // BOARD_HEIGHT

    # Create a copy of the original image to draw the grid
    grid_image = img_orig.copy()

    # Draw vertical lines for the grid
    for i in range(1, BOARD_WIDTH):
        cv2.line(grid_image, (i * cell_width, 0), (i * cell_width, img_h), (0, 255, 0), 1)

    # Draw horizontal lines for the grid
    for i in range(1, BOARD_HEIGHT):
        cv2.line(grid_image, (0, i * cell_height), (img_w, i * cell_height), (0, 255, 0), 1)

    # Display the image with the grid
    cv2.imwrite('grid_image_with_cells.png', grid_image)

    # Initialize the grid
    grid = np.zeros((BOARD_HEIGHT, BOARD_WIDTH))

    # Constants for color detection for black and white pieces RGB bounds!
    BLACK_LOWER_HSV = np.array([0, 0, 0])
    BLACK_UPPER_HSV = np.array([110, 110, 110])
    WHITE_LOWER_HSV = np.array([150, 150, 150])
    WHITE_UPPER_HSV = np.array([255, 255, 255])
    
    # Create masks for white and black pieces
    white_mask = cv2.inRange(bilateral_filtered_image, WHITE_LOWER_HSV, WHITE_UPPER_HSV)
    cv2.imwrite('white_mask.png', white_mask)

    black_mask = cv2.inRange(bilateral_filtered_image, BLACK_LOWER_HSV, BLACK_UPPER_HSV)
    cv2.imwrite('black_mask.png', black_mask)

    #im_with_black_mask = cv2.bitwise_and(bilateral_filtered_image, bilateral_filtered_image,mask=black_mask)
    #im_with_bw_mask = cv2.bitwise_and(im_with_black_mask, white_mask)

    game_pieces_mask = black_mask + white_mask
    cv2.imwrite('game_pieces_mask.png', game_pieces_mask)

    not_game_pieces_mask = cv2.bitwise_not(game_pieces_mask)

    # Need to mask away the red, green, and yellow colors as well
    RED_MASK_LOWER = np.array([80, 0, 0])
    RED_MASK_UPPER = np.array([150, 50, 50])
    GREEN_MASK_LOWER = np.array([0, 110, 0])
    GREEN_MASK_UPPER = np.array([30, 150, 30])
    YELLOW_MASK_LOWER = np.array([216, 216, 39])
    YELLOW_MASK_UPPER = np.array([255, 255, 77])
    red_mask = cv2.inRange(bilateral_filtered_image, RED_MASK_LOWER, RED_MASK_UPPER)
    cv2.imwrite('red_mask.png', red_mask)
    green_mask = cv2.inRange(bilateral_filtered_image, GREEN_MASK_LOWER, GREEN_MASK_UPPER)
    cv2.imwrite('green_mask.png', green_mask)
    yellow_mask = cv2.inRange(bilateral_filtered_image, YELLOW_MASK_LOWER, YELLOW_MASK_UPPER)
    
    # Background color masking
    BOARD_BACKGROUND_LOWER = np.array([70, 60, 50])
    BOARD_BACKGROUND_UPPER = np.array([90, 255, 255])
    
    # Create a mask for the background color of the board
    background_mask = cv2.inRange(bilateral_filtered_image, BOARD_BACKGROUND_LOWER, BOARD_BACKGROUND_UPPER)

    # Invert the background mask to remove the background color
    background_mask = cv2.bitwise_not(background_mask)

    # Combine masks to create a single mask representing areas to be removed
    unwanted_mask = not_game_pieces_mask + red_mask + green_mask + yellow_mask
    cv2.imwrite('unwanted_mask.png', unwanted_mask)

    # Apply the background mask to remove the background color
    result_image = cv2.bitwise_and(bilateral_filtered_image, bilateral_filtered_image, mask=background_mask)
    result_image = cv2.bitwise_and(bilateral_filtered_image, bilateral_filtered_image, mask=unwanted_mask)

    # Combine the result with the game pieces mask to identify the game pieces
    result_image = cv2.bitwise_and(bilateral_filtered_image, bilateral_filtered_image, mask=unwanted_mask)
    result_image = cv2.bitwise_or(result_image, result_image, mask=not_game_pieces_mask)
    cv2.imwrite('result_image.png', result_image)


    # Function to check if the majority of the pixels in a masked area are of the color of the mask
    def is_color_dominant(mask):
        # Count the non-zero (white) pixels in the mask
        white_pixels = cv2.countNonZero(mask)
        # Calculate the percentage of white pixels
        white_area_ratio = white_pixels / mask.size
        # If the white area covers more than 30% of the mask, we consider the color to be dominant
        return white_area_ratio > 0.3


    def process_cell(img, x_start, y_start, width, height):
        # Crop the cell from the image
        cell_img = img[y_start:y_start + height, x_start:x_start + width]
        cv2.imwrite('cell_img.png', cell_img)

        # Create masks for white and black pieces
        white_mask_cell = cv2.inRange(cell_img, WHITE_LOWER_HSV, WHITE_UPPER_HSV)
        cv2.imwrite('white_mask_cell.png', white_mask_cell)
        black_mask_cell = cv2.inRange(cell_img, BLACK_LOWER_HSV, BLACK_UPPER_HSV)
        cv2.imwrite('black_mask_cell.png', black_mask_cell)

        if is_color_dominant(white_mask_cell):
            print("White piece detected")
            return -1  # White 
        elif is_color_dominant(black_mask_cell):
            print("Black piece detected")
            return 1  # Black 
        else:
            # The space is empty
            return 0

    # CONTINUE PROCESS FRAME
    # Analyze each cell and update the grid
    for row in range(BOARD_HEIGHT):
        for col in range(BOARD_WIDTH):
            x_start = col * cell_width
            y_start = row * cell_height
            grid[row, col] = process_cell(bilateral_filtered_image, x_start, y_start, cell_width, cell_height)

    
    return grid

# determines if there is motion in the frame
backSub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)

# Could be better...
def is_motion(previous_frame, current_frame, threshold=10):
    frame_delta = cv2.absdiff(previous_frame, current_frame)
    thresholded = cv2.threshold(frame_delta, 25, 255, cv2.THRESH_BINARY)[1]
    motion_level = np.sum(thresholded)
    return motion_level > threshold


# Need to compelte find_board_bounds
def find_board_bounds(frame, threshold=50):
    x, y = 0, 0  # Top left corner
    w, h = 1154, 1152  # Width and height calculated from bottom right - top left
    
    return x, y, w, h


def board_is_full(grid):
    for row in grid:
        for cell in row:
            if cell == 0:
                return False
    return True


def check_winner(grid):
    rows, cols = len(grid), len(grid[0])
    grid_array = np.array(grid)

    # If the board is not full then we have not found a winner
    if (not board_is_full(grid_array)):
        return 0
    
    # If the board is full then we need to check for a winner
    
    # Win condition
    # 1. count the number of black and white tiles
    # 2. return the color & count with more tiles

    black_count = 0
    white_count = 0
    for row in grid_array:
        for cell in grid_array:
            if cell == 1:
                black_count += 1
            elif cell == -1:
                white_count += 1

    # If black has more tiles then black wins
    if black_count > white_count:
        return 1
    elif white_count > black_count:
        return -1
    else:
        return 0


def extract_frames(video_path, skip_frames=20):
    cap = cv2.VideoCapture(video_path)
    previous_position_string = None
    output_strings = []  # List to store output strings

    ret, previous_frame = cap.read()
    if not ret:
        output_strings.append("Error: Cannot read frame from video.")
        cap.release()
        return output_strings
    
    previous_frame = cv2.cvtColor(previous_frame, cv2.COLOR_BGR2GRAY)
    previous_frame = cv2.GaussianBlur(previous_frame, (21, 21), 0)
    previous_position_string = '------------------------------------------'
    p, frame_count = 0, 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)
        if frame_count % skip_frames == 0:
            if not is_motion(previous_frame, gray):
                board_array = process_frame(frame)
                current_position_string = grid_to_position_string(board_array)
                
                # There is a change between one game state and the next
                if current_position_string != previous_position_string:
                    output_strings.append(f'p={str((p % 2) + 1)}_{current_position_string}')
                    p += 1

                # result = check_winner(board_array)
                # if result == -1:
                #     output_strings.append("Player 2 WHITE wins!")
                #     break
                # elif result == 1:
                #     output_strings.append("Player 1 BLACK wins!")
                #     break

                previous_position_string = current_position_string
            previous_frame = gray
        frame_count += 1

    cap.release()
    return output_strings

print(extract_frames('uploads/othellogameplay.mov'))

Black piece detected
White piece detected
White piece detected
Black piece detected
Black piece detected
White piece detected
White piece detected
Black piece detected
Black piece detected
White piece detected
White piece detected
Black piece detected
Black piece detected
White piece detected
White piece detected
Black piece detected
Black piece detected
Black piece detected
Black piece detected
White piece detected
Black piece detected
Black piece detected
Black piece detected
Black piece detected
White piece detected
Black piece detected
Black piece detected
Black piece detected
Black piece detected
White piece detected
Black piece detected
White piece detected
Black piece detected
White piece detected
Black piece detected
White piece detected
Black piece detected
White piece detected
Black piece detected
White piece detected
Black piece detected
White piece detected
Black piece detected
White piece detected
Black piece detected
White piece detected
Black piece detected
White piece d

# Old Image Circle Code

In [ ]:
# Convert the cell image to grayscale
        # gray_img = cv2.cvtColor(cell_img, cv2.COLOR_BGR2GRAY)
        # cv2.imwrite('gray_img.png', gray_img)

        # Apply Hough Circle Transform to find circles in the grayscale image
        # circles = cv2.HoughCircles(gray_img, cv2.HOUGH_GRADIENT, dp=1.2, minDist=20,
        #                            param1=50, param2=30, minRadius=0, maxRadius=0)

        # if circles is not None:
        #     circles = np.round(circles[0, :]).astype("int")
        #     for (x, y, r) in circles:
        #         # Ensure the crop coordinates are within the image bounds
        #         x, y, r = int(x), int(y), int(r)
        #         x1, y1, x2, y2 = max(0, x - r), max(0, y - r), min(width, x + r), min(height, y + r)

        #         # Crop the circle from the cell_img
        #         circle_img = cell_img[y1:y2, x1:x2]

        #         # If the circle_img is empty, skip to the next circle
        #         if circle_img.size == 0:
        #             continue

        #         # Convert to HSV and create masks for black and white
        #         hsv_circle_img = cv2.cvtColor(circle_img, cv2.COLOR_BGR2HSV)
        #         cv2.imwrite('hsv_circle_img.png', hsv_circle_img)

        #         black_mask_circle = cv2.inRange(hsv_circle_img, BLACK_LOWER_HSV, BLACK_UPPER_HSV)
        #         cv2.imwrite('black_mask_circle.png', black_mask_circle)
        #         white_mask_circle = cv2.inRange(hsv_circle_img, WHITE_LOWER_HSV, WHITE_UPPER_HSV)
        #         cv2.imwrite('white_mask_circle.png', white_mask_circle)

        #         if is_color_dominant(white_mask_circle):
        #             print("White piece detected")
        #             return -1  # White 
        #         elif is_color_dominant(black_mask_circle):
        #             print("Black piece detected")
        #             return 1  # Black 
        #         else:
        #             # The space is empty
        #             return 0
